# Scraping BestFightOdds.com

In [4]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil      
import datetime
from scipy.stats import norm
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Excel-fixed data:

Initially, I matched up events in excel.

In [7]:
# load excel-fixed data
fight_odds_urls = pd.read_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv')

print(fight_odds_urls.shape)
fight_odds_urls.head(3)

(532, 11)


,event_title,event_url,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,event_name,event_date,event_id,bfo_url
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,NaN,NaN,NaN,NaN
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,NaN,NaN,NaN,NaN
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,NaN,NaN,NaN,NaN


In [8]:
# get all event urls
event_urls = fight_odds_urls['event_url'].unique()
print(f' {len(event_urls)} events')

# get all event urls
event_urls = fight_odds_urls['event_url'].unique()
# split by /, keep last
event_urls = [url.split('/')[-1] for url in event_urls]

 520 events


In [13]:

def get_last_events_ufcstats():
    """
    Retrieves information about completed UFC events from the UFC Stats website.
    Returns a DataFrame with columns: event_name, event_date, event_url, event_id.
    """
    url = 'http://www.ufcstats.com/statistics/events/completed'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Extract event links and names
    events = soup.find_all('a', class_='b-link b-link_style_black')
    event_links = [link.get('href') for link in events]
    event_names = [link.text.strip() for link in events]

    # Extract event dates
    event_dates = soup.find_all('span', class_='b-statistics__date')
    event_dates = [date.text.strip() for date in event_dates][1:]

    # Create DataFrame
    df = pd.DataFrame({'event_name': event_names, 'event_date': event_dates, 'event_url': event_links})
    df['event_date'] = pd.to_datetime(df['event_date'])
    df = df.sort_values(by='event_date', ascending=False).reset_index(drop=True)
    df['event_id'] = df['event_url'].apply(lambda x: x.split('/')[-1])
    
    return df

In [14]:
last_events = get_last_events_ufcstats()
last_events

,event_name,event_date,event_url,event_id
0,UFC Fight Night: Vera vs. Sandhagen,2023-03-25,http://www.ufcstats.com/event-details/aec273fc...,aec273fcb765330d
1,UFC 286: Edwards vs. Usman 3,2023-03-18,http://www.ufcstats.com/event-details/e4bb7e48...,e4bb7e483c4ad318
2,UFC Fight Night: Yan vs. Dvalishvili,2023-03-11,http://www.ufcstats.com/event-details/35080a7f...,35080a7f406f9ab3
3,UFC 285: Jones vs. Gane,2023-03-04,http://www.ufcstats.com/event-details/1ccff7f0...,1ccff7f0cfdf85eb
4,UFC Fight Night: Muniz vs. Allen,2023-02-25,http://www.ufcstats.com/event-details/806975e1...,806975e1b4f47b27
5,UFC Fight Night: Andrade vs. Blanchfield,2023-02-18,http://www.ufcstats.com/event-details/f21a3d68...,f21a3d68fb9df387
6,UFC 284: Makhachev vs. Volkanovski,2023-02-11,http://www.ufcstats.com/event-details/01dd4cdc...,01dd4cdc2446f665
7,UFC Fight Night: Lewis vs. Spivac,2023-02-04,http://www.ufcstats.com/event-details/d26394fc...,d26394fc0e8e880a
8,UFC 283: Teixeira vs. Hill,2023-01-21,http://www.ufcstats.com/event-details/5717efc6...,5717efc6f271cd52
9,UFC Fight Night: Strickland vs. Imavov,2023-01-14,http://www.ufcstats.com/event-details/2e2cdb6e...,2e2cdb6e9eb84bb9


In [16]:
last_events_urls = last_events['event_id'].unique()
# get events  in last_events that are not in fight_odds_urls
new_events = [event for event in last_events_urls if event not in event_urls]
new_events

[]

Identify Events to Download

In [17]:
missing_events = last_events[last_events['event_id'].isin(new_events)]
# strip event names
missing_events['event_name'] = missing_events['event_name'].apply(lambda x: x.strip())
missing_events

,event_name,event_date,event_url,event_id


Find BestFightOdds url for event

In [95]:
driver = webdriver.Chrome()

In [96]:
def get_bfo_url(event_name):
    """
    Input is event name from missing_events
    Output is url from bestfightodds.com
    """
    filtered_df = missing_events[missing_events['event_name'] == event_name]
    driver.get('https://www.google.com/')
    # wait 1 sec
    time.sleep(1)

    # search 'site: bestfightodds.com'
    search = driver.find_element_by_name('q')
    search.send_keys('site:bestfightodds.com ' + '"events"' +' ' + filtered_df['event_name'].iloc[0])
    search.send_keys(Keys.RETURN)
    
    # get the first link
    first_link = driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div/div/div/div[1]/a')
    first_link.click()
    
    # get the url
    url = driver.current_url
    return url

In [97]:
missing_events['event_name'][0]

'UFC Fight Night: Vera vs. Sandhagen'

In [98]:
get_bfo_url(missing_events['event_name'][0])

'https://www.bestfightodds.com/events/ufc-fight-night-222-2724'

In [99]:
# for each event in missing_events, get the bfo url and add it to the dataframe with lambda apply

bfo_urls = []
for event in missing_events['event_name']:
    try:
        bfo_url = get_bfo_url(event)
        bfo_urls.append(bfo_url)
    except:
        bfo_urls.append('error')
        print(f'error with {event}')

missing_events['bfo_url'] = bfo_urls
missing_events

error with UFC 285: Jones vs. Gane
error with UFC 282: Blachowicz vs. Ankalaev


C:\Users\Travis\AppData\Local\Temp\ipykernel_30856\2417590038.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_events['bfo_url'] = bfo_urls


,event_name,event_date,event_url,event_id,bfo_url
0,UFC Fight Night: Vera vs. Sandhagen,2023-03-25,http://www.ufcstats.com/event-details/aec273fc...,aec273fcb765330d,https://www.bestfightodds.com/events/ufc-fight...
1,UFC 286: Edwards vs. Usman 3,2023-03-18,http://www.ufcstats.com/event-details/e4bb7e48...,e4bb7e483c4ad318,https://www.bestfightodds.com/events/ufc-278-u...
2,UFC Fight Night: Yan vs. Dvalishvili,2023-03-11,http://www.ufcstats.com/event-details/35080a7f...,35080a7f406f9ab3,https://www.bestfightodds.com/events/ufc-fight...
3,UFC 285: Jones vs. Gane,2023-03-04,http://www.ufcstats.com/event-details/1ccff7f0...,1ccff7f0cfdf85eb,error
4,UFC Fight Night: Muniz vs. Allen,2023-02-25,http://www.ufcstats.com/event-details/806975e1...,806975e1b4f47b27,https://www.bestfightodds.com/events/ufc-fight...
5,UFC Fight Night: Andrade vs. Blanchfield,2023-02-18,http://www.ufcstats.com/event-details/f21a3d68...,f21a3d68fb9df387,https://www.bestfightodds.com/events/ufc-fight...
6,UFC 284: Makhachev vs. Volkanovski,2023-02-11,http://www.ufcstats.com/event-details/01dd4cdc...,01dd4cdc2446f665,https://www.bestfightodds.com/events/ufc-284-m...
7,UFC Fight Night: Lewis vs. Spivac,2023-02-04,http://www.ufcstats.com/event-details/d26394fc...,d26394fc0e8e880a,https://www.bestfightodds.com/events/ufc-fight...
8,UFC 283: Teixeira vs. Hill,2023-01-21,http://www.ufcstats.com/event-details/5717efc6...,5717efc6f271cd52,https://www.bestfightodds.com/events/ufc-283-2635
9,UFC Fight Night: Strickland vs. Imavov,2023-01-14,http://www.ufcstats.com/event-details/2e2cdb6e...,2e2cdb6e9eb84bb9,https://www.bestfightodds.com/events/ufc-fight...


If there are any errors here, it may be necessary to fix manually, like I did for the two matches below:

In [100]:
# # Add https://www.bestfightodds.com/events/ufc-282-2621 to UFC 282: Blachowicz vs. Ankalaev
# missing_events['bfo_url'][11] = 'https://www.bestfightodds.com/events/ufc-282-2621'

# # add jones vs gane
# missing_events['bfo_url'][3] = 'https://www.bestfightodds.com/events/ufc-285-2738'


C:\Users\Travis\AppData\Local\Temp\ipykernel_30856\3251729377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_events['bfo_url'][11] = 'https://www.bestfightodds.com/events/ufc-282-2621'
C:\Users\Travis\AppData\Local\Temp\ipykernel_30856\3251729377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_events['bfo_url'][3] = 'https://www.bestfightodds.com/events/ufc-285-2738'


In [101]:
# rename bfo_url to event_odds_url
missing_events = missing_events.rename(columns={'bfo_url': 'event_odds_url'})
# rename event_name to event_title
missing_events = missing_events.rename(columns={'event_name': 'event_title'})
# drop event_id
missing_events = missing_events.drop(columns=['event_id'])

In [102]:
# update fight_odds_urls with missing_events
fight_odds_urls = pd.concat([fight_odds_urls, missing_events], axis=0)
fight_odds_urls = fight_odds_urls.reset_index(drop=True)
fight_odds_urls.head(3)

,event_title,event_url,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,event_name,event_date,event_id,bfo_url
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,NaN,NaT,NaN,NaN
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,NaN,NaT,NaN,NaN
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,NaN,NaT,NaN,NaN
3,UFC Fight Night: Rodriguez vs. Lemos,http://ufcstats.com/event-details/756f45905fb2...,https://www.bestfightodds.com/events/ufc-fight...,Rodriguez vs. Lemos,Rodriguez vs Lemos,rodriguez,lemos,NaN,NaT,NaN,NaN
4,UFC Fight Night: Kattar vs. Allen,http://ufcstats.com/event-details/be5aab761c40...,https://www.bestfightodds.com/events/ufc-fight...,Kattar vs. Allen,Kattar vs Allen,kattar,allen,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
527,UFC Fight Night: Lewis vs. Spivac,http://www.ufcstats.com/event-details/d26394fc...,https://www.bestfightodds.com/events/ufc-fight...,NaN,NaN,NaN,NaN,NaN,2023-02-04,NaN,NaN
528,UFC 283: Teixeira vs. Hill,http://www.ufcstats.com/event-details/5717efc6...,https://www.bestfightodds.com/events/ufc-283-2635,NaN,NaN,NaN,NaN,NaN,2023-01-21,NaN,NaN
529,UFC Fight Night: Strickland vs. Imavov,http://www.ufcstats.com/event-details/2e2cdb6e...,https://www.bestfightodds.com/events/ufc-fight...,NaN,NaN,NaN,NaN,NaN,2023-01-14,NaN,NaN
530,UFC Fight Night: Cannonier vs. Strickland,http://www.ufcstats.com/event-details/56ec5895...,https://www.bestfightodds.com/events/ufc-fight...,NaN,NaN,NaN,NaN,NaN,2022-12-17,NaN,NaN


In [103]:
# save to csv
fight_odds_urls.to_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv', index=False)

#### Scraping BFO via Requests & Headers

In [104]:
# you can scrape bestfightodds by pretending to be a browser
# you will get a 404 error if you don't do this

url = 'https://www.bestfightodds.com/events/ufc-282-2621'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
dfs = pd.read_html(r.text)
dfs[0]

,Jan Brachowicz
0,Magomed Ankalaev
1,Over 1½ rounds
2,Under 1½ rounds
3,Over 2½ rounds
4,Under 2½ rounds
...,...
1725,Under 7½ fights go the distance
1726,Over 8½ fights go the distance
1727,Under 8½ fights go the distance
1728,Over 9½ fights go the distance


## Scraping BFO Data

In [105]:
fight_odds_urls = pd.read_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv')

,event_title,event_url,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2,event_name,event_date,event_id,bfo_url
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland,NaN,NaN,NaN,NaN
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba,NaN,NaN,NaN,NaN
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira,NaN,NaN,NaN,NaN
3,UFC Fight Night: Rodriguez vs. Lemos,http://ufcstats.com/event-details/756f45905fb2...,https://www.bestfightodds.com/events/ufc-fight...,Rodriguez vs. Lemos,Rodriguez vs Lemos,rodriguez,lemos,NaN,NaN,NaN,NaN
4,UFC Fight Night: Kattar vs. Allen,http://ufcstats.com/event-details/be5aab761c40...,https://www.bestfightodds.com/events/ufc-fight...,Kattar vs. Allen,Kattar vs Allen,kattar,allen,NaN,NaN,NaN,NaN


In [106]:
all_event_odds_urls = fight_odds_urls.event_odds_url.unique()
len(all_event_odds_urls)

501

In [107]:
len('https://www.bestfightodds.com/events/')

37

In [108]:
def get_odds(url):
        # you can scrape by pretending to be a browser
        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        r = requests.get(url, headers = header)
        dfs = pd.read_html(r.text)
        data = dfs[1]
        data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
        data['event_odds_url'] = url
        event_name = url.split('/')[4]
        data['event_id'] = event_name
        data.to_csv('data/final/events/odds_per_event/' + event_name + '.csv', index=False)
        return data



In [109]:
get_odds(all_event_odds_urls[0]).head(2)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,Bet365,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id
0,Kevin Holland,-170▲,-185▼,-195▼,-200▼,-166▼,NaN,-200▼,-175▲,-188▲,NaN,-200▼,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607
1,Stephen Thompson,+145▼,+150▲,+165▲,+160▲,+140▲,NaN,+160▲,+150▼,+150▼,NaN,+170▲,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607


In [110]:
# drop nan
all_event_odds_urls = all_event_odds_urls[~pd.isnull(all_event_odds_urls)]
all_event_odds_urls

array(['https://www.bestfightodds.com/events/ufc-fight-night-214-2607',
       'https://www.bestfightodds.com/events/ufc-fight-night-215-2633',
       'https://www.bestfightodds.com/events/ufc-281-2529',
       'https://www.bestfightodds.com/events/ufc-fight-night-214ting-championship-2612',
       'https://www.bestfightodds.com/events/ufc-fight-night-213-2606',
       'https://www.bestfightodds.com/events/ufc-281-2586',
       'https://www.bestfightodds.com/events/ufc-fight-night-212-2579',
       'https://www.bestfightodds.com/events/ufc-vegas-57-2489',
       'https://www.bestfightodds.com/events/ufc-fight-night-211-2569',
       'https://www.bestfightodds.com/events/ufc-279-2541',
       'https://www.bestfightodds.com/events/ufc-fight-night-gane-vs-tuivasa-2518',
       'https://www.bestfightodds.com/events/ufc-278-usman-vs-edwards-2-2545',
       'https://www.bestfightodds.com/events/ufc-fight-night-vera-vs-cruz-2552',
       'https://www.bestfightodds.com/events/ufc-on-espn-santo

In [111]:
# Initialize a counter variable to keep track of the number of URLs processed
n = 0

# Initialize lists to store URLs that resulted in errors and those that were processed successfully
errors = []
complete = []

# Get a list of filenames from the 'data/final/events/odds_per_event/' directory
files = os.listdir('data/final/events/odds_per_event/')

# Extract the file identifiers (excluding the file extension) to store them in 'file_list'
file_list = [n[:-4] for n in files]

# Loop through each URL in the list of URLs pointing to event odds pages
for url in all_event_odds_urls:
    # Extract the unique identifier from the URL (excluding the file extension)
    url_file_name = url.split('/')[-1]
    url_file_name2 = url_file_name[:-4]
    
    # Check if the unique identifier is already in the list of processed files
    if url_file_name not in file_list:
        try:
            # Attempt to get odds data from the URL using the 'get_odds' function
            get_odds(url)
            
            # Print a message indicating the progress (number of URLs processed out of total)
            print(f'{n} / {len(all_event_odds_urls)}')
            
            # Increment the counter variable
            n += 1
        except:
            # Print an error message if an exception occurs while getting odds data
            print(f'ERROR! {n} / {len(all_event_odds_urls)}')
            
            # Append the problematic URL to the 'errors' list
            errors.append(url)
            
            # Increment the counter variable
            n += 1
    else:
        # If the unique identifier is already in 'file_list', print a message indicating that it exists
        print(f'{url_file_name} already exists')
        
        # Append the unique identifier to the 'complete' list
        complete.append(url_file_name)

# Print the total number of errors encountered during the execution
print(f' total errors: {len(errors)}')


ufc-fight-night-214-2607 already exists
ufc-fight-night-215-2633 already exists
ufc-281-2529 already exists
ufc-fight-night-214ting-championship-2612 already exists
ufc-fight-night-213-2606 already exists
ufc-281-2586 already exists
ufc-fight-night-212-2579 already exists
ufc-vegas-57-2489 already exists
ufc-fight-night-211-2569 already exists
ufc-279-2541 already exists
ufc-fight-night-gane-vs-tuivasa-2518 already exists
ufc-278-usman-vs-edwards-2-2545 already exists
ufc-fight-night-vera-vs-cruz-2552 already exists
ufc-on-espn-santos-vs-hill-2534 already exists
ufc-277-pena-vs-nunes-2-2517 already exists
ufc-fight-night-blaydes-vs-aspinall-2519 already exists
ufc-on-abc-ortega-vs-rodriguez-2524 already exists
ufc-on-espn-dos-anjos-vs-fiziev-2501 already exists
ufc-276-adesanya-vs-cannonier-2478 already exists
ufc-on-espn-tsarukyan-vs-gamrot-2574 already exists
ufc-fight-night-kattar-vs-emmett-2523 already exists
ufc-275-teixeira-vs-prochazka-2561 already exists
ufc-fight-night-211-244

### Append Together

In [112]:
# Get list of all files in 'data/final/events/odds_per_event/' directory
files = os.listdir('data/final/events/odds_per_event/')

# Create an empty list to store all the event odds data frames
all_event_odds = []

# Loop through each file in the directory
for file in files:
    # Read the csv file and store it as a pandas data frame
    df = pd.read_csv('data/final/events/odds_per_event/' + file)
    # Append the data frame to the list of all event odds data frames
    all_event_odds.append(df)

# Concatenate all the data frames in the list into a single data frame
all_event_odds = pd.concat(all_event_odds)

# Print the shape (dimensions) of the final data frame and display the first few rows
print(all_event_odds.shape)
all_event_odds.head()

Add Event Names & URLs to Event_Odds df

In [115]:
def get_event_title(event_odds_url):
    # first, filter the fight_odds_urls dataframe to only include rows with a matching event_odds_url
    data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
    # next, extract the event_title value from the filtered dataframe
    event_name = data.event_title.values[0]
    # finally, return the event_title value
    return event_name

def get_ufcstats_url(event_odds_url):
    # similar to the previous function, filter the fight_odds_urls dataframe and extract the event_url value
    data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
    event_url = data.event_url.values[0]
    # return the event_url value
    return event_url

In [117]:
all_event_odds['event_name'] = all_event_odds.apply(lambda row: get_event_title(row['event_odds_url']), axis=1)

In [119]:
all_event_odds['event_ufcstats_url'] = all_event_odds.apply(lambda row: get_ufcstats_url(row['event_odds_url']), axis=1)

In [120]:
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [121]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V1.csv', index=False)

In [122]:
all_event_odds['5D'] = all_event_odds['5D'].str[:-1]
all_event_odds['5D'] = all_event_odds['5D'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [123]:
all_event_odds['Ref'] = all_event_odds['Ref'].str[:-1]
all_event_odds['Ref'] = all_event_odds['Ref'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [124]:
# Choose columns to keep
cols = ['fighter', '5D', 'Ref', 'event_odds_url', 'event_ufcstats_url', 'event_id', 'event_name']
all_event_odds = all_event_odds[cols]
all_event_odds.head()

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [125]:
# get shape
all_event_odds.shape

(346015, 7)

In [126]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V2.csv', index=False)


## Summary
- ALL ODDS by fighter are available at:
  - data/ufc_BestFightOdds/All_Event_Odds_by_Fighter.csv
- You can combine this with UfcStatsData easily using 'event_ufcstats_url' 

# Add All Events Df

Saved in final/events

In [127]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')

In [128]:
# events are located in data/ufc_stats/events2

folder = 'data/ufc_stats/events2/'

# aggregate all events into one dataframe
all_events = []

for file in os.listdir(folder):
    df = pd.read_csv(folder + file)
    all_events.append(df)

all_events = pd.concat(all_events)

print(all_events.shape)
all_events.head()

(5730, 19)


,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,1,110,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...
1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,0,74,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...
2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,0,51,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...
3,3,win,Middleweight,KO/TKO Punches,3,1:09,Roman Kopylov,Alessio Di Chirico,1,0,59,39,0,0,0,0,4,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/1a191251...
4,4,win,Featherweight,U-DEC,3,5:00,William Gomis,Jarno Errens,0,0,32,20,3,0,0,1,5,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/398db400...


In [129]:
all_events.to_csv('data/final/events/All_Events_V1.csv', index=False)

In [130]:
all_events = pd.read_csv('data/final/events/All_Events_V1.csv')

# Add Odd Changes to Odds

In [131]:
# create a function to extract odds data from a given URL
def get_odds(url):
    
    # add headers to avoid getting blocked while making requests
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    
    # make a GET request to the URL with the provided headers
    r = requests.get(url, headers=headers)
    
    # scrape the HTML tables present in the response content
    dfs = pd.read_html(r.text)
    
    # extract the relevant data from the second table
    data = dfs[1]
    
    # rename the first column as "fighter" for better readability
    data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
    
    # add the event odds URL as a new column in the dataframe
    data['event_odds_url'] = url
    
    # extract the event name from the URL and add it as a new column in the dataframe
    event_name = url.split('/')[4]
    data['event_id'] = event_name
    
    # uncomment the following line to save the extracted data as a CSV file
    # data.to_csv('data/ufc_BestFightOdds/odds_by_event/' + event_name + '.csv', index=False)
    
    # return the dataframe with the extracted odds data
    return data

In [132]:
def get_bfodds(event_url):
    # scrape the changes in odds by event url
    
    driver = webdriver.Chrome()
    driver.get(event_url)

    # click show all
    driver.find_element_by_class_name('event-swing-expand').click()
    #wait 2 seconds
    time.sleep(2)

    event_swing_container = driver.find_element_by_id('event-swing-container')
    event_swing_container.text


    # the first half of the text is the odds changes, the seconds half are the names. Lets match them up. 
    # first split the text by new line
    event_swing_container.text.split('\n')

    # get the names. You can identify them because they will be more than 5 characters long
    names = [x for x in event_swing_container.text.split('\n') if len(x) > 5]
    names

    # get the odds changes
    odds_changes = [x for x in event_swing_container.text.split('\n') if '%' in x]
    odds_changes

    # get the event name
    event_name = driver.find_element_by_class_name('table-header')
    # name is h1 in event_name
    name = event_name.find_element_by_tag_name('h1')
    event = name.text

    # get the url
    url = driver.current_url


    # create a dataframe
    df = pd.DataFrame({'names': names, 'odds_changes': odds_changes, 'event': event, 'event_url': url})
    
    #close the driver
    driver.close()

    return df

In [133]:
all_odds_eventlist = all_event_odds['event_odds_url'].unique().tolist()

In [ ]:
# Initialize lists to keep track of completed, failed, and newly completed events
already_complete = []
failed = []
newly_complete = []

# Iterate through all events in the all_odds_eventlist
for url in all_odds_eventlist:
    # Extract the event name from the URL
    event_name = url.split('/')[-1]
    
    # Check if the event has already been completed
    if str(event_name) + '.csv' in os.listdir('data/final/odds/odds_changes/'):
        print('already done with ' + url.split('/')[-1])
        already_complete.append(event_name)
        continue
    else:
        try:
            # Get the odds data for the event
            df = get_bfodds(url)
            name = url.split('/')[-1]
            
            # Save the odds data to a CSV file
            df.to_csv('data/final/odds/odds_changes/' + name + '.csv', index=False)
            newly_complete.append(name)
            print('done with ' + name)
        except:
            # If there's an error, add the event to the failed list
            name = url.split('/')[-1]
            failed.append(name)
            print('failed on ' + name)
            continue

# Print the summary of completed, newly completed, and failed events
print("")
print('already done with ' + str(len(already_complete)) + ' events')
print('newly done with ' + str(len(newly_complete)) + ' events')
print('failed on ' + str(len(failed)) + ' events')

In [148]:
# aggregate all odds changes into one dataframe
all_odds_changes = []

for file in os.listdir('data/final/odds/odds_changes/'):
    df = pd.read_csv('data/final/odds/odds_changes/' + file)
    # append
    all_odds_changes.append(df)

all_odds_changes = pd.concat(all_odds_changes)

print(all_odds_changes.shape)
all_odds_changes.to_csv('data/final/aggregates/All_Odds_Changes_V1.csv', index=False)
all_odds_changes

(11649, 4)


,names,odds_changes,event,event_url
0,Deron Winn,-61%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
1,Chuck Liddell,-30%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
2,Gleison Tibau,-28%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
3,Ricardo Palacios,-14%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
4,Kenneth Bergh,+3%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
...,...,...,...,...
19,Daniel Santos,+34%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
20,Brendan Allen,+35%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
21,Mike Davis,+55%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
22,Philipe Lins,+60%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...


### Edit Event_odds 

Get rid of alternate lines

In [150]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')
all_event_odds.head(3)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
346010,Any other result,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
346011,Either fighter wins by unanimous decision,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
346012,Any other result,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
346013,Either fighter wins by split/majority decision,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


In [151]:
all_options = all_event_odds['fighter'].value_counts()
all_options = pd.DataFrame(all_options)
all_options.reset_index(inplace=True)
delete_options = all_options.head(120)
delete_options

,index,fighter
0,Any other result,95876
1,Fight goes to decision,4754
2,Fight doesn't go to decision,4754
3,Fight is a draw,4748
4,Fight is not a draw,4748
...,...,...
115,Fight won't go 2:30 round 2,200
116,Under 1½ fights go the distance,185
117,Over 1½ fights go the distance,185
118,Exactly one fight goes the distance,185


In [152]:
delete_options_list = delete_options['index'].tolist()

In [153]:
# delete rows when fighter is in delete_options_list
all_event_odds = all_event_odds[~all_event_odds['fighter'].isin(delete_options_list)]
all_event_odds.head(2)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [154]:
# delete any rows where fighter has more than 3 words
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('round')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('decision')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('submission')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('knockout')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('draw')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('scorecards')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('majority')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('Not')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('wins')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('losses')]
all_event_odds


,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
345608,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
345743,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
345744,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
345879,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


# Add Odds Change to Odds

In [155]:
def get_odds_change_from_db(event_odds_url, fighter):
    """
    Get the odds change from the all_odds_changes dataframe.
    """

    try:
        odds_change = all_odds_changes[(all_odds_changes['event_url'] == event_odds_url)
                                       & (all_odds_changes['names'] == fighter)]['odds_changes'].values[0]
        
        return odds_change
    
    except:
        return np.nan

In [156]:
# test
get_odds_change_from_db('https://www.bestfightodds.com/events/ufc-100-137', 'Brock Lesnar')

'-31%'

In [157]:
# Create odds_change column
all_event_odds["odds_change"] = all_event_odds.apply(
    lambda row: get_odds_change_from_db(row["event_odds_url"], row["fighter"]),
    axis=1
)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31%
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+59%
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-6%
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+4%
...,...,...,...,...,...,...,...,...
345608,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-32%
345743,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-6%
345744,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,+4%
345879,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-22%


In [158]:
# get rid of %
all_event_odds['odds_change'] = all_event_odds['odds_change'].str.replace('%', '')
# turn into float
all_event_odds['odds_change'] = all_event_odds['odds_change'].astype(float)
all_event_odds.head(3)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31.0
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,59.0
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN


In [159]:
# save to csv
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_WithChange.csv', index=False)

In [160]:
print(f' This was last run: {datetime.datetime.now()}')

 This was last run: 2023-04-04 12:38:44.271543
